In [1]:
import pandas as pd
import numpy as np

In [2]:
pd.options.display.max_columns = None

In [3]:
sixteen = pd.read_csv('nba_odds_16.csv')
seventeen = pd.read_csv('nba_odds_17.csv')
eighteen = pd.read_csv('nba_odds_18.csv')
nineteen = pd.read_csv('nba_odds_19.csv')
twenty = pd.read_csv('nba_odds_20.csv')
twentyone = pd.read_csv('nba_odds_21.csv')
twentytwo = pd.read_csv('nba_odds_22.csv')
twentythree = pd.read_csv('nba_odds_23.csv')

SEASONS = [sixteen,seventeen,eighteen,nineteen,twenty,twentyone,twentytwo,twentythree]

In [4]:
count = 2016
for s in SEASONS:
    s['season'] = count
    count+=1


In [5]:
def clean_date(date):
    if date >= 1000:
        return pd.to_datetime(date + 150000, format = "%y%m%d")
    else:
        return pd.to_datetime(date + 160000, format = "%y%m%d")
    
sixteen.Date = sixteen.Date.apply(clean_date)

In [6]:
def clean_date(date):
    if date >= 1000:
        return pd.to_datetime(date + 160000, format = "%y%m%d")
    else:
        return pd.to_datetime(date + 170000, format = "%y%m%d")
    
seventeen.Date = seventeen.Date.apply(clean_date)

In [7]:
def clean_date(date):
    if date >= 1000:
        return pd.to_datetime(date + 170000, format = "%y%m%d")
    else:
        return pd.to_datetime(date + 180000, format = "%y%m%d")
    
eighteen.Date = eighteen.Date.apply(clean_date)

In [8]:
def clean_date(date):
    if date >= 1000:
        return pd.to_datetime(date + 180000, format = "%y%m%d")
    else:
        return pd.to_datetime(date + 190000, format = "%y%m%d")
    
nineteen.Date = nineteen.Date.apply(clean_date)

In [9]:
def clean_date(date):
    if date >= 1022:
        return pd.to_datetime(date + 190000, format = "%y%m%d")
    else:
        return pd.to_datetime(date + 200000, format = "%y%m%d")

twenty.Date = twenty.Date.apply(clean_date)

In [10]:
def clean_date(date):
    if date >= 1000:
        return pd.to_datetime(date + 200000, format = "%y%m%d")
    else:
        return pd.to_datetime(date + 210000, format = "%y%m%d")

twentyone.Date = twentyone.Date.apply(clean_date)

In [11]:
def clean_date(date):
    if date >= 1000:
        return pd.to_datetime(date + 210000, format = "%y%m%d")
    else:
        return pd.to_datetime(date + 220000, format = "%y%m%d")

twentytwo.Date = twentytwo.Date.apply(clean_date)

In [12]:
def clean_date(date):
    if date >= 1000:
        return pd.to_datetime(date + 220000, format = "%y%m%d")
    else:
        return pd.to_datetime(date + 230000, format = "%y%m%d")

twentythree.Date = twentythree.Date.apply(clean_date)

In [13]:
odds = pd.concat(list(SEASONS))

In [14]:
base_cols = ['Date','VH','Team','Final','Close','ML','season']
odds = odds[base_cols]

In [15]:
odds.columns = odds.columns.str.lower()
odds = odds.reset_index(drop = True)
odds.index = odds.index+1

In [16]:
# I need to map Team names into abreviated version

team_abr = {'Atlanta':'ATL','Brooklyn':'BKN','Boston':'BOS','Charlotte':'CHA','Chicago':'CHI','Cleveland':'CLE',
            'Dallas':'DAL','Denver':'DEN','Detroit':'DET', 'GoldenState':'GSW','Golden State':'GSW','Houston':'HOU',
            'Indiana':'IND','LAClippers':'LAC','LA Clippers':'LAC','LALakers':'LAL','LA Lakers':'LAL','Memphis':'MEM',
            'Miami':'MIA','Milwaukee':'MIL','Minnesota':'MIN','NewOrleans':'NOP','New Orleans':'NOP','NewYork':'NYK',
            'New York':'NYK','OklahomaCity':'OKC', 'Okla City':'OKC','Orlando':'ORL','Philadelphia':'PHI',
            'Phoenix':'PHX','Portland':'POR','Sacramento':'SAC','SanAntonio':'SAS','San Antonio':'SAS',
            'Toronto':'TOR','Utah':'UTA','Washington':'WAS'}

odds.team = odds.team.map(team_abr)

In [17]:
odds['spread'] = None
odds['ou'] = None
odds['fav'] = None

In [18]:
odds['close'] = odds['close'].replace('pk','0').replace('PK','0').astype(float)

In [19]:
def over_under_first(i):
    odds.loc[i,'ou'] = odds.loc[i,'close']
    odds.loc[i+1,'ou'] = odds.loc[i,'close']
    odds.loc[i,'spread'] = odds.loc[i+1,'close']
    odds.loc[i+1,'spread'] = odds.loc[i+1,'close']
    
    odds.loc[i,'fav'] = 0
    odds.loc[i+1,'fav'] = 1
    

def spread_first(i):
    odds.loc[i,'ou'] = odds.loc[i+1,'close']
    odds.loc[i+1,'ou'] = odds.loc[i+1,'close']
    odds.loc[i,'spread'] = odds.loc[i,'close']
    odds.loc[i+1,'spread'] = odds.loc[i,'close']
    
    odds.loc[i,'fav'] = 1
    odds.loc[i+1,'fav'] = 0

In [20]:
for i in range(1,len(odds.index),2):
    if odds.loc[i,'close']>100:
        over_under_first(i)
    else:
        spread_first(i)             

In [21]:
del odds['vh']
del odds['final']
del odds['close']
del odds['ml']
del odds['season']

In [22]:
odds.to_csv('odds_combined')

In [23]:
odds.tail()

,date,team,spread,ou,fav
19096,2023-01-16,MIN,4.0,232.0,1
19097,2023-01-16,PHX,12.5,231.0,0
19098,2023-01-16,MEM,12.5,231.0,1
19099,2023-01-16,HOU,7.5,236.5,0
19100,2023-01-16,LAL,7.5,236.5,1


# Start Here

In [24]:
import pandas as pd
import numpy as np
odds = pd.read_csv('odds_combined',index_col=0)

In [25]:
df = pd.read_csv("nba_games_updated.csv",index_col=0)

In [26]:
df.date = pd.to_datetime(df.date)
odds.date = pd.to_datetime(odds.date)

In [27]:
merged = df.merge(right=odds, on=['date','team'],how='left')

In [28]:
total_points = merged['total']+merged['total_opp']
merged['ou_bet'] = np.where(total_points<=merged['ou'],0,1)

In [29]:
# Calculate spread_bet column

for i in range(0,len(merged.index)):
    if merged.loc[i,'fav']==1:
        if merged.loc[i,'total'] - merged.loc[i,'total_opp'] > merged.loc[i,'spread']:
            merged.loc[i,'spread_bet'] =1
        else:
            merged.loc[i,'spread_bet'] =0
    else:
        if merged.loc[i,'total_opp'] - merged.loc[i,'total'] < merged.loc[i,'spread']: 
            merged.loc[i,'spread_bet'] =1
        else:
            merged.loc[i,'spread_bet'] =0

In [30]:
merged.drop(merged[pd.isnull(merged.ou)].index,inplace=True)
merged.drop(merged[pd.isnull(merged.spread)].index,inplace=True)
merged.drop(merged[(merged.spread>25) | (merged.ou<100)].index,inplace=True)
merged.loc[merged[merged.fav==1].index,'spread'] = merged[merged.fav==1][['spread']]*-1
merged[['team','date','fav','total','total_opp','spread','spread_bet','ou','ou_bet']].head()

merged.to_csv('nba_games_test.csv')

# Merged/nba_games_test is old games with sports betting columns

In [31]:
merged = pd.read_csv('nba_games_test.csv',index_col=0)

In [32]:
merged.sort_values('date').tail()

,mp,mp.1,fg,fga,fg%,3p,3pa,3p%,ft,fta,ft%,orb,drb,trb,ast,stl,blk,tov,pf,pts,+/-,ts%,efg%,3par,ftr,orb%,drb%,trb%,ast%,stl%,blk%,tov%,usg%,ortg,drtg,mp_max,mp_max.1,fg_max,fga_max,fg%_max,3p_max,3pa_max,3p%_max,ft_max,fta_max,ft%_max,orb_max,drb_max,trb_max,ast_max,stl_max,blk_max,tov_max,pf_max,pts_max,+/-_max,ts%_max,efg%_max,3par_max,ftr_max,orb%_max,drb%_max,trb%_max,ast%_max,stl%_max,blk%_max,tov%_max,usg%_max,ortg_max,drtg_max,team,total,home,index_opp,mp_opp,mp_opp.1,fg_opp,fga_opp,fg%_opp,3p_opp,3pa_opp,3p%_opp,ft_opp,fta_opp,ft%_opp,orb_opp,drb_opp,trb_opp,ast_opp,stl_opp,blk_opp,tov_opp,pf_opp,pts_opp,+/-_opp,ts%_opp,efg%_opp,3par_opp,ftr_opp,orb%_opp,drb%_opp,trb%_opp,ast%_opp,stl%_opp,blk%_opp,tov%_opp,usg%_opp,ortg_opp,drtg_opp,mp_max_opp,mp_max_opp.1,fg_max_opp,fga_max_opp,fg%_max_opp,3p_max_opp,3pa_max_opp,3p%_max_opp,ft_max_opp,fta_max_opp,ft%_max_opp,orb_max_opp,drb_max_opp,trb_max_opp,ast_max_opp,stl_max_opp,blk_max_opp,tov_max_opp,pf_max_opp,pts_max_opp,+/-_max_opp,ts%_max_opp,efg%_max_opp,3par_max_opp,ftr_max_opp,orb%_max_opp,drb%_max_opp,trb%_max_opp,ast%_max_opp,stl%_max_opp,blk%_max_opp,tov%_max_opp,usg%_max_opp,ortg_max_opp,drtg_max_opp,team_opp,total_opp,home_opp,season,date,won,spread,ou,fav,ou_bet,spread_bet
19312,240.0,240.0,39.0,80.0,0.488,9.0,28.0,0.321,26.0,33.0,0.788,13.0,30.0,43.0,22.0,5.0,4.0,15.0,19.0,113.0,NaN,0.598,0.544,0.350,0.413,34.2,90.9,60.6,56.4,5.3,8.3,13.7,100.0,118.7,108.2,NaN,NaN,9.0,22.0,0.800,5.0,10.0,0.50,7.0,7.0,1.0,7.0,7.0,11.0,11.0,2.0,1.0,4.0,4.0,30.0,16.0,0.820,0.800,0.667,2.0,23.9,56.1,34.7,46.1,3.5,5.1,27.9,29.4,174.0,113.0,CLE,113,1,0,240.0,240.0,38.0,75.0,0.507,10.0,27.0,0.370,17.0,22.0,0.773,3.0,25.0,28.0,26.0,9.0,1.0,17.0,23.0,103.0,NaN,0.608,0.573,0.360,0.293,9.1,65.8,39.4,68.4,9.5,1.9,16.7,100.0,108.2,118.7,NaN,NaN,8.0,20.0,0.778,3.0,9.0,1.0,7.0,9.0,1.0,2.0,11.0,13.0,5.0,4.0,1.0,4.0,4.0,25.0,8.0,0.849,0.833,1.000,2.000,8.3,39.7,25.1,31.3,7.1,2.6,42.9,36.6,152.0,128.0,NOP,103,0,2023,2023-01-16,True,-7.5,223.5,1.0,0,1.0
19311,240.0,240.0,38.0,75.0,0.507,10.0,27.0,0.370,17.0,22.0,0.773,3.0,25.0,28.0,26.0,9.0,1.0,17.0,23.0,103.0,NaN,0.608,0.573,0.360,0.293,9.1,65.8,39.4,68.4,9.5,1.9,16.7,100.0,108.2,118.7,NaN,NaN,8.0,20.0,0.778,3.0,9.0,1.00,7.0,9.0,1.0,2.0,11.0,13.0,5.0,4.0,1.0,4.0,4.0,25.0,8.0,0.849,0.833,1.000,2.0,8.3,39.7,25.1,31.3,7.1,2.6,42.9,36.6,152.0,128.0,NOP,103,0,1,240.0,240.0,39.0,80.0,0.488,9.0,28.0,0.321,26.0,33.0,0.788,13.0,30.0,43.0,22.0,5.0,4.0,15.0,19.0,113.0,NaN,0.598,0.544,0.350,0.413,34.2,90.9,60.6,56.4,5.3,8.3,13.7,100.0,118.7,108.2,NaN,NaN,9.0,22.0,0.800,5.0,10.0,0.5,7.0,7.0,1.0,7.0,7.0,11.0,11.0,2.0,1.0,4.0,4.0,30.0,16.0,0.820,0.800,0.667,2.000,23.9,56.1,34.7,46.1,3.5,5.1,27.9,29.4,174.0,113.0,CLE,113,1,2023,2023-01-16,False,7.5,223.5,0.0,0,0.0
19327,240.0,240.0,43.0,84.0,0.512,19.0,40.0,0.475,14.0,15.0,0.933,4.0,33.0,37.0,29.0,6.0,5.0,15.0,17.0,119.0,NaN,0.657,0.625,0.476,0.179,11.1,78.6,47.4,67.4,6.0,11.1,14.2,100.0,118.8,131.8,NaN,NaN,11.0,17.0,1.000,5.0,10.0,1.00,6.0,6.0,1.0,2.0,8.0,8.0,9.0,3.0,4.0,4.0,4.0,30.0,7.0,1.250,1.250,0.667,2.0,9.5,39.6,24.9,51.9,4.5,11.8,34.7,93.5,247.0,139.0,IND,119,0,1,240.0,240.0,48.0,91.0,0.527,23.0,46.0,0.500,13.0,15.0,0.867,9.0,32.0,41.0,32.0,9.0,1.0,13.0,14.0,132.0,NaN,0.676,0.654,0.505,0.165,21.4,88.9,52.6,66.7,9.0,2.3,11.8,100.0,131.8,118.8,NaN,NaN,13.0,19.0,1.000,5.0,8.0,1.0,4.0,4.0,1.0,2.0,9.0,11.0,11.0,3.0,1.0,3.0,3.0,35.0,18.0,1.000,1.000,1.000,1.000,9.8,35.5,20.0,45.4,6.4,3.2,25.8,26.6,216.0,125.0,MIL,132,1,2023,2023-01-16,False,5.0,228.5,0.0,1,0.0
19318,240.0,240.0,42.0,86.0,0.488,14.0,35.0,0.400,20.0,23.0,0.870,6.0,30.0,36.0,31.0,7.0,2.0,14.0,21.0,118.0,NaN,0.614,0.570,0.407,0.267,15.0,76.9,45.6,73.8,6.9,4.3,12.7,100.0,116.7,125.6,NaN,NaN,9.0,20.0,0.778,4.0,12.0,0.75,11.0,12.0,1.0,3.0,8.0,11.0,10.0,2.0,2.0,6.0,4.0,32.0,17.0,0.889,0.889,0.800,0.8,17.2,26.4,21.8,46.8,3.4,5.6,21.6,34.3,153.0,133.0,WAS,118,1,0,240.0,240.0,45.0,91.0,0.495,18.0,44.0,0.409,19.0,21.0,0.905,9.0,34.0,43.0,28.0,8.0,1.0,12.0,2

# Scrape odds from today, create df to update main df

In [33]:
import pandas as pd
test = pd.read_html('https://www.teamrankings.com/nba/odds/')

In [34]:
team_abr = {'Atlanta':'ATL','Brooklyn':'BKN','Boston':'BOS','Charlotte':'CHA','Chicago':'CHI','Cleveland':'CLE',
            'Dallas':'DAL','Denver':'DEN','Detroit':'DET', 'GoldenState':'GSW','Golden State':'GSW','Houston':'HOU',
            'Indiana':'IND','LAClippers':'LAC','LA Clippers':'LAC','LALakers':'LAL','LA Lakers':'LAL','Memphis':'MEM',
            'Miami':'MIA','Milwaukee':'MIL','Minnesota':'MIN','NewOrleans':'NOP','New Orleans':'NOP','NewYork':'NYK',
            'New York':'NYK','OklahomaCity':'OKC', 'Okla City':'OKC','Orlando':'ORL','Philadelphia':'PHI',
            'Phoenix':'PHX','Portland':'POR','Sacramento':'SAC','SanAntonio':'SAS','San Antonio':'SAS',
            'Toronto':'TOR','Utah':'UTA','Washington':'WAS'}

In [35]:
# Read in today's odds, clean the data and combine into one df
from datetime import date

spread = pd.concat([test[2],test[3]])
total = pd.concat([test[4],test[5]])

total = pd.concat([total[0].str.replace('\sat','vs').str.replace('\(N\)','').str.strip().str.split('vs.',expand=True),total[1]],axis=1)
spread = pd.concat([spread.iloc[:,0].str.replace('\sat','vs').str.replace('\(N\)','').str.strip().str.split('vs.',expand=True),spread.iloc[:,1]],axis=1)

spread.columns = ['tm1','tm2','spread']
total.columns = ['tm1','tm2','ou']

spread['tm1'] = spread['tm1'].str.strip().map(team_abr)
spread['tm2'] = spread['tm2'].str.strip().map(team_abr)

total['tm1'] = total['tm1'].str.strip().map(team_abr)
total['tm2'] = total['tm2'].str.strip().map(team_abr)

spread = spread.sort_values('tm1').reset_index(drop=True)
total = total.sort_values('tm1').reset_index(drop=True)

#Combine spread and total into 1 df, add 'fav' column and 'date_tonight'
combined = pd.concat([spread,total['ou']],axis=1)
combined['fav'] = 1
combined['date_tonight'] = date.today()
combined = combined.reset_index(drop=True)

# Get new df swapping tm1 and tm 2
comb = combined[['tm2','spread','ou']]
comb['fav']=0
comb.columns = ['tm1','spread','ou','fav']

odds_tonight = pd.concat([combined,comb]).reset_index(drop=True)
odds_tonight['spread'] = odds_tonight['spread']*-1
del odds_tonight['tm2']
del odds_tonight['date_tonight']
odds_tonight.columns = ['team','spreadO','ouO','favO']


odds_tonight.to_csv('odds_tonight.csv')

/var/folders/fz/0282wvb93rn0lm_p0nscw1sm0000gn/T/ipykernel_20763/1859340060.py:7: FutureWarning: The default value of regex will change from True to False in a future version.
  total = pd.concat([total[0].str.replace('\sat','vs').str.replace('\(N\)','').str.strip().str.split('vs.',expand=True),total[1]],axis=1)
/var/folders/fz/0282wvb93rn0lm_p0nscw1sm0000gn/T/ipykernel_20763/1859340060.py:8: FutureWarning: The default value of regex will change from True to False in a future version.
  spread = pd.concat([spread.iloc[:,0].str.replace('\sat','vs').str.replace('\(N\)','').str.strip().str.split('vs.',expand=True),spread.iloc[:,1]],axis=1)
/var/folders/fz/0282wvb93rn0lm_p0nscw1sm0000gn/T/ipykernel_20763/1859340060.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-vie

In [36]:
odds_tonight

,team,spreadO,ouO,favO
0,CHI,2.0,234.0,1
1,CLE,3.5,220.5,1
2,DAL,7.0,226.0,1
3,DEN,1.0,230.0,1
4,LAC,5.0,230.0,1
5,MIA,4.5,215.5,1
6,PHX,6.0,227.0,1
7,IND,2.0,234.0,0
8,NYK,3.5,220.5,0
9,WAS,7.0,226.0,0
